In [1]:
using Serialization: serialize, deserialize
using MPI
MPI.Init()

MPI.ThreadLevel(2)

In [2]:
comm = MPI.COMM_WORLD
println("Hello world, I am $(MPI.Comm_rank(comm) + 1) of $(MPI.Comm_size(comm))")
MPI.Barrier(comm)

Hello world, I am 1 of 1


In [3]:
MPI.Comm

MPI.Comm

In [4]:
ccall((:braid_Hello, "./libbraid.so"), Cint, ());

Hello world!


In [5]:
# Data structures required by XBraid
mutable struct braid_app
    comm::MPI.Comm      # MPI communicator
    bufsize::Integer    # default size of a braid_vector allocation
    vec_refs::IdDict    # dictionary which stores global references to
                        # braid_vectors, to keep them from being garbage
                        # collected while they only exist as C pointers
                        # keys are objectid hashes, and values are references
end

mutable struct braid_vector
    data
end

vec = braid_vector(zeros(4))
vec_ser = IOBuffer()
serialize(vec_ser, vec)
app = braid_app(MPI.COMM_WORLD, vec_ser.size, IdDict())

braid_app(MPI.Comm(1140850688), 69, IdDict{Any, Any}())

In [6]:
function assign_like_c!(dest :: T, src :: T) :: T where {T}
       fields = fieldnames(T)
       setfield!.((dest,), fields, getfield.((src,), fields))
       return dest
end

function my_pack!(_app, vec, buffer::Ptr{Cvoid})::Cint
    println("pack")
    vec_ref = unsafe_pointer_to_objref(vec)
    data_arr = unsafe_wrap(Vector{UInt8}, Base.unsafe_convert(Ptr{UInt8}, buffer), app.bufsize)
    buff = IOBuffer(data_arr, write=true, maxsize=app.bufsize)
    serialize(buff, vec_ref)
    return 0
end

function my_unpack!(_app, vec, buffer::Ptr{Cvoid})::Cint
    println("unpack")
    vec_ref = unsafe_pointer_to_objref(vec)
    data_arr = unsafe_wrap(Vector{UInt8}, Base.unsafe_convert(Ptr{UInt8}, buffer), app.bufsize)
    buff = IOBuffer(data_arr, read=true, maxsize=app.bufsize)
    show(buff)
    println(typeof(vec))
    # unpack the buffer into a julia struct, then shallow copy the struct
    assign_like_c!(vec_ref, deserialize(buff))

    # vec = Base.unsafe_convert(Ptr{Cvoid}, Ref(vec_ref))
    return 0
end

function my_buffsize!(_app, size::Ptr{Cint})::Cint
    println("size")
    unsafe_store!(size, app.bufsize)
    return 0
end

function my_init!(_app, t::Real, u_ptr::Ptr{Ptr{braid_vector}})
    println("init")
    u = zeros(4)
    # Since app is in the global scope, u will survive garbage collection even after it goes out of scope here
    app.vec_refs[objectid(u)] = u # store reference to u
    unsafe_store!(u_ptr, pointer_from_objref(new_vec))
    return 0
end

function my_free!(_app, u::Ptr{braid_vector})
    println("free")
    u_ref = unsafe_pointer_to_objref(u)
    # removing the global reference to u will allow garbage collection to delete it
    pop!(app.vec_refs, objectid(u_ref))
    u = C_NULL
    return 0
end

function my_sum!(_app, alpha::Real, x::Ptr{braid_vector}, beta::Real, y::Ptr{braid_vector})
    x_ref = unsafe_pointer_to_objref(x)
    y_ref = unsafe_pointer_to_objref(y)
    # @. vectorizes every operation in this line, meaning this assignment is in-place!
    @. y_ref.data = alpha*x_ref.data + beta*y_ref.data
    return 0
end

function my_clone!(_app, u::Ptr{braid_vector}, v_ptr::Ptr{Ptr{braid_vector}})
    u_ref = unsafe_pointer_to_objref(u)
    v_ref = deepcopy(u_ref) # this allocates v_ref and copies u_ref
    # need to add reference to ref_ids
    app.vec_refs[objectid(v_ref)] = v_ref
    unsafe_store!(v_ptr, pointer_from_objref(v_ref))
    return 0
end

pack_c = @cfunction(my_pack!, Cint, (Ptr{Cvoid}, Ptr{Cvoid}, Ptr{Cvoid}))
unpack_c = @cfunction(my_unpack!, Cint, (Ptr{Cvoid}, Ptr{Cvoid}, Ptr{Cvoid}))
buffsize_c = @cfunction(my_buffsize!, Cint, (Ptr{Cvoid}, Ptr{Cint}))

Ptr{Nothing} @0x00007fb0b016d920

In [7]:
u = Ref([1, 2, 3, 4])
v = [2, 2, 3, 3]
u_ptr = Base.Libc.malloc(8)
u_ptr = Base.unsafe_convert(Ptr{Ptr{braid_vector}}, u_ptr)

Ptr{Ptr{braid_vector}} @0x00000000023d46b0

In [9]:
ref_ids = IdDict()

IdDict{Any, Any}()

In [10]:
function init()
    a = braid_vector([1, 2, 3])
    # add reference to IdDict to maintain global reference,
    # or else garbage collection may destroy this later!
    ref_ids[objectid(a)] = a
    return pointer_from_objref(a)
end

function free!(a_ptr)
    a = unsafe_pointer_to_objref(a_ptr)
    println(ref_ids[objectid(a)])
    pop!(ref_ids, objectid(a))
    a_ptr = C_NULL
end

free! (generic function with 1 method)

In [11]:
a_ptr = init()
b_ptr = init()
show(ref_ids)
GC.gc()
free!(a_ptr)
free!(b_ptr)

IdDict{Any, Any}(0x8644c409cf4b8a4b => braid_vector(

[1, 2, 3]), 0xc15fe162c581f458 => braid_vector([1, 2, 3]))braid_vector

([1, 2, 3])
braid_vector([1, 2, 3])


Ptr{Nothing} @0x0000000000000000

In [12]:
ref_ids

IdDict{Any, Any}()